In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc

In [2]:
connection_parameters = {
    "account": "xe85544.east-us-2.azure",
    "user": "kx",
    "password": os.environ['SNOWFLAKE_TEMP_PASSWORD'],
    "role": "SYSADMIN",  # optional
    "warehouse": "KX",  # medium snowpark-optimized
    "database": "KX",
    "schema": "BRUNO",
}

session = Session.builder.configs(connection_parameters).create()

In [4]:
@sproc(packages=['snowflake-snowpark-python','pykx'])
def kx(session: Session) -> dict:

    import pykx as kx
    
    #Import trades data
    trades_sdf = session2.table('TRADES')

    select_columns = ['"TTime"', '"Symbol"', '"Trade Volume"', '"Trade Price"']
    trades = trades_sdf.select(select_columns).where('"Symbol" = \'AAPL\'').order_by('"TTime"').toPandas()
    trades.columns = ['time', 'symbol', 'tvolume', 'tprice']
    trades.reset_index(drop=True, inplace=True)

    #import quote data 
    quotes_sdf = session2.table('QUOTES')

    select_columns = ['"TTime"', '"Symbol"', '"Bid_Price"', '"Bid_Size"', '"Offer_Price"', '"Offer_Size"']
    quotes = quotes_sdf.select(select_columns).where('"Symbol" = \'AAPL\'').order_by('"TTime"').toPandas()
    quotes.columns = ['time', 'symbol', 'qbid_price', 'qbid_size', 'qask_price', 'qask_size']
    quotes.reset_index(drop=True, inplace=True)
    
    #Q
    kx.q['trades'] = trades
    kx.q['quotes'] = quotes

    kx.q('trades: update `g#symbol, `s#time from trades')
    kx.q('quotes: update `g#symbol, `s#time from quotes')
    kx.q('rez:aj[`symbol`time; trades; quotes]')
    kx.q('rez2: update qmid_price: (qbid_price + qask_price) % 2 from rez')
    kx.q('rez2: update diff:tprice - qmid_price from rez2')
    kx.q('rez2: aj[`symbol`time; rez2; select symbol, time: time - 0D00:00:10, mid10:(qbid_price + qask_price) % 2 from quotes]')

    return 'SUCCESS'


In [6]:
kx()

{'result': {'pandas_to_kx': '0:00:02.641630', 'q': '0:00:00.412932'}}